In [0]:
#Spark Session
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Spark Introduction").getOrCreate()

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
#Employee Data & Schema  --> list of lists
data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","Female","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [0]:
# Create emp DataFrame
emp = spark.createDataFrame(data,schema)

In [0]:
#Check number of partitions
emp.rdd.getNumPartitions()

Out[6]: 8

In [0]:

emp.show()  #action

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [0]:
#Schema for emp

emp.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)



In [0]:
# CASTING column using pyspark datafram APIs provided by functions module
# SELECT employee_id, name , age, cast(salary as double) as salary from emp
from pyspark.sql.functions import col, cast
emp_casted = emp.select("employee_id", "name", "age", col("salary").cast("double"))

emp_casted.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- salary: double (nullable = true)



In [0]:
# ADDing column tax which is 20% of the column salary
# SELECT employee_id, name, age, salary, (salary * 0.2) as tax from emp_casted

emp_taxed = emp_casted.withColumn("tax", col("salary") * 0.2)
emp_taxed.show()

#OR
from pyspark.sql.functions import expr
emp_taxed_expr = emp_casted.select("*", expr("salary * 0.2").alias("tax"))
emp_taxed_expr.show()

#OR

emp_taxed_e2 = emp_casted.selectExpr("*", "salary * 0.2 as tax")
emp_taxed_e2.show()
#All the above methods gives you same result


+-----------+-------------+---+-------+-------+
|employee_id|         name|age| salary|    tax|
+-----------+-------------+---+-------+-------+
|        001|     John Doe| 30|50000.0|10000.0|
|        002|   Jane Smith| 25|45000.0| 9000.0|
|        003|    Bob Brown| 35|55000.0|11000.0|
|        004|    Alice Lee| 28|48000.0| 9600.0|
|        005|    Jack Chan| 40|60000.0|12000.0|
|        006|    Jill Wong| 32|52000.0|10400.0|
|        007|James Johnson| 42|70000.0|14000.0|
|        008|     Kate Kim| 29|51000.0|10200.0|
|        009|      Tom Tan| 33|58000.0|11600.0|
|        010|     Lisa Lee| 27|47000.0| 9400.0|
|        011|   David Park| 38|65000.0|13000.0|
|        012|   Susan Chen| 31|54000.0|10800.0|
|        013|    Brian Kim| 45|75000.0|15000.0|
|        014|    Emily Lee| 26|46000.0| 9200.0|
|        015|  Michael Lee| 37|63000.0|12600.0|
|        016|  Kelly Zhang| 30|49000.0| 9800.0|
|        017|  George Wang| 34|57000.0|11400.0|
|        018|    Nancy Liu| 29|50000.0|1

In [0]:
# Literals
# SELECT employee_id, name, age, salary, tax, 1 as columnOne, 'Two' as columnTwo from emp_taxed
from pyspark.sql.functions import lit

emp_new_cols = emp_taxed.withColumn("columnOne",lit(1)).withColumn("columnTwo", lit("Two"))
emp_new_cols.show()

+-----------+-------------+---+-------+-------+---------+---------+
|employee_id|         name|age| salary|    tax|columnOne|columnTwo|
+-----------+-------------+---+-------+-------+---------+---------+
|        001|     John Doe| 30|50000.0|10000.0|        1|      Two|
|        002|   Jane Smith| 25|45000.0| 9000.0|        1|      Two|
|        003|    Bob Brown| 35|55000.0|11000.0|        1|      Two|
|        004|    Alice Lee| 28|48000.0| 9600.0|        1|      Two|
|        005|    Jack Chan| 40|60000.0|12000.0|        1|      Two|
|        006|    Jill Wong| 32|52000.0|10400.0|        1|      Two|
|        007|James Johnson| 42|70000.0|14000.0|        1|      Two|
|        008|     Kate Kim| 29|51000.0|10200.0|        1|      Two|
|        009|      Tom Tan| 33|58000.0|11600.0|        1|      Two|
|        010|     Lisa Lee| 27|47000.0| 9400.0|        1|      Two|
|        011|   David Park| 38|65000.0|13000.0|        1|      Two|
|        012|   Susan Chen| 31|54000.0|10800.0| 

In [0]:
# Renaming columns
# SELECT employee_id as emp_if, name, age, salary. tax, columnOne, columntwo from emp_new_cols
emp1 = emp_new_cols.withColumnRenamed("employee_id","emp_id")
emp1.show()

+------+-------------+---+-------+-------+---------+---------+
|emp_id|         name|age| salary|    tax|columnOne|columnTwo|
+------+-------------+---+-------+-------+---------+---------+
|   001|     John Doe| 30|50000.0|10000.0|        1|      Two|
|   002|   Jane Smith| 25|45000.0| 9000.0|        1|      Two|
|   003|    Bob Brown| 35|55000.0|11000.0|        1|      Two|
|   004|    Alice Lee| 28|48000.0| 9600.0|        1|      Two|
|   005|    Jack Chan| 40|60000.0|12000.0|        1|      Two|
|   006|    Jill Wong| 32|52000.0|10400.0|        1|      Two|
|   007|James Johnson| 42|70000.0|14000.0|        1|      Two|
|   008|     Kate Kim| 29|51000.0|10200.0|        1|      Two|
|   009|      Tom Tan| 33|58000.0|11600.0|        1|      Two|
|   010|     Lisa Lee| 27|47000.0| 9400.0|        1|      Two|
|   011|   David Park| 38|65000.0|13000.0|        1|      Two|
|   012|   Susan Chen| 31|54000.0|10800.0|        1|      Two|
|   013|    Brian Kim| 45|75000.0|15000.0|        1|   

In [0]:
# Column names with spaces
# SELECT employee_id as emp_id, name, age, salary, tax, columnOne, columnTwo as 'Column Two' from emp_new_cols

emp_2 = emp_new_cols.withColumnRenamed('columnTwo', 'Column Two')
emp_2.show()

#Spark does accept column names with spaces but it is never recommended to name any columns with spaces as it might throw an error in production system

+-----------+-------------+---+-------+-------+---------+----------+
|employee_id|         name|age| salary|    tax|columnOne|Column Two|
+-----------+-------------+---+-------+-------+---------+----------+
|        001|     John Doe| 30|50000.0|10000.0|        1|       Two|
|        002|   Jane Smith| 25|45000.0| 9000.0|        1|       Two|
|        003|    Bob Brown| 35|55000.0|11000.0|        1|       Two|
|        004|    Alice Lee| 28|48000.0| 9600.0|        1|       Two|
|        005|    Jack Chan| 40|60000.0|12000.0|        1|       Two|
|        006|    Jill Wong| 32|52000.0|10400.0|        1|       Two|
|        007|James Johnson| 42|70000.0|14000.0|        1|       Two|
|        008|     Kate Kim| 29|51000.0|10200.0|        1|       Two|
|        009|      Tom Tan| 33|58000.0|11600.0|        1|       Two|
|        010|     Lisa Lee| 27|47000.0| 9400.0|        1|       Two|
|        011|   David Park| 38|65000.0|13000.0|        1|       Two|
|        012|   Susan Chen| 31|540

In [0]:
# Remove column
emp_col_dropped = emp_new_cols.drop("columnTwo")
emp_col_dropped.show()

+-----------+-------------+---+-------+-------+---------+
|employee_id|         name|age| salary|    tax|columnOne|
+-----------+-------------+---+-------+-------+---------+
|        001|     John Doe| 30|50000.0|10000.0|        1|
|        002|   Jane Smith| 25|45000.0| 9000.0|        1|
|        003|    Bob Brown| 35|55000.0|11000.0|        1|
|        004|    Alice Lee| 28|48000.0| 9600.0|        1|
|        005|    Jack Chan| 40|60000.0|12000.0|        1|
|        006|    Jill Wong| 32|52000.0|10400.0|        1|
|        007|James Johnson| 42|70000.0|14000.0|        1|
|        008|     Kate Kim| 29|51000.0|10200.0|        1|
|        009|      Tom Tan| 33|58000.0|11600.0|        1|
|        010|     Lisa Lee| 27|47000.0| 9400.0|        1|
|        011|   David Park| 38|65000.0|13000.0|        1|
|        012|   Susan Chen| 31|54000.0|10800.0|        1|
|        013|    Brian Kim| 45|75000.0|15000.0|        1|
|        014|    Emily Lee| 26|46000.0| 9200.0|        1|
|        015| 

In [0]:
# Remove more than one column

emp_col_dropped = emp_new_cols.drop("columnTwo", "columnOne")
emp_col_dropped.show()


+-----------+-------------+---+-------+-------+
|employee_id|         name|age| salary|    tax|
+-----------+-------------+---+-------+-------+
|        001|     John Doe| 30|50000.0|10000.0|
|        002|   Jane Smith| 25|45000.0| 9000.0|
|        003|    Bob Brown| 35|55000.0|11000.0|
|        004|    Alice Lee| 28|48000.0| 9600.0|
|        005|    Jack Chan| 40|60000.0|12000.0|
|        006|    Jill Wong| 32|52000.0|10400.0|
|        007|James Johnson| 42|70000.0|14000.0|
|        008|     Kate Kim| 29|51000.0|10200.0|
|        009|      Tom Tan| 33|58000.0|11600.0|
|        010|     Lisa Lee| 27|47000.0| 9400.0|
|        011|   David Park| 38|65000.0|13000.0|
|        012|   Susan Chen| 31|54000.0|10800.0|
|        013|    Brian Kim| 45|75000.0|15000.0|
|        014|    Emily Lee| 26|46000.0| 9200.0|
|        015|  Michael Lee| 37|63000.0|12600.0|
|        016|  Kelly Zhang| 30|49000.0| 9800.0|
|        017|  George Wang| 34|57000.0|11400.0|
|        018|    Nancy Liu| 29|50000.0|1

In [0]:
# Filter data
# Select employee_id as emp_id, name, age, salary, tax from emp_col_dropped where tax > 1000

emp_filtered = emp_col_dropped.where("tax > 1000")
emp_filtered.show()

+-----------+-------------+---+-------+-------+
|employee_id|         name|age| salary|    tax|
+-----------+-------------+---+-------+-------+
|        001|     John Doe| 30|50000.0|10000.0|
|        002|   Jane Smith| 25|45000.0| 9000.0|
|        003|    Bob Brown| 35|55000.0|11000.0|
|        004|    Alice Lee| 28|48000.0| 9600.0|
|        005|    Jack Chan| 40|60000.0|12000.0|
|        006|    Jill Wong| 32|52000.0|10400.0|
|        007|James Johnson| 42|70000.0|14000.0|
|        008|     Kate Kim| 29|51000.0|10200.0|
|        009|      Tom Tan| 33|58000.0|11600.0|
|        010|     Lisa Lee| 27|47000.0| 9400.0|
|        011|   David Park| 38|65000.0|13000.0|
|        012|   Susan Chen| 31|54000.0|10800.0|
|        013|    Brian Kim| 45|75000.0|15000.0|
|        014|    Emily Lee| 26|46000.0| 9200.0|
|        015|  Michael Lee| 37|63000.0|12600.0|
|        016|  Kelly Zhang| 30|49000.0| 9800.0|
|        017|  George Wang| 34|57000.0|11400.0|
|        018|    Nancy Liu| 29|50000.0|1

In [0]:
# LIMIT data
# Select employee_id as emp_id, name, age, salary, tax from emp_filtered limit 5

emp_limit = emp_filtered.limit(5)
emp_limit.show()

+-----------+----------+---+-------+-------+
|employee_id|      name|age| salary|    tax|
+-----------+----------+---+-------+-------+
|        001|  John Doe| 30|50000.0|10000.0|
|        002|Jane Smith| 25|45000.0| 9000.0|
|        003| Bob Brown| 35|55000.0|11000.0|
|        004| Alice Lee| 28|48000.0| 9600.0|
|        005| Jack Chan| 40|60000.0|12000.0|
+-----------+----------+---+-------+-------+



In [0]:
# IMPORTANT
#Adding multiple columns

columns = {
    "tax" : col("salary")*0.2,
    "numOne" : lit(1),
    "colTwo" : lit("Two")
}

emp_final = emp.withColumns(columns)
emp_final.show()

+-----------+-------------+-------------+---+------+------+----------+-------+------+------+
|employee_id|department_id|         name|age|gender|salary| hire_date|    tax|numOne|colTwo|
+-----------+-------------+-------------+---+------+------+----------+-------+------+------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|10000.0|     1|   Two|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15| 9000.0|     1|   Two|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|11000.0|     1|   Two|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30| 9600.0|     1|   Two|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|12000.0|     1|   Two|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|10400.0|     1|   Two|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|14000.0|     1|   Two|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|